# LSTM Test - Kaza - PCA

In [1]:
import pandas as pd
import sys
from pathlib import Path
sys.path.append('../..')
from src import config
from src.utils.dataset import TrafficDataset
from src.utils.model import TrafficPredictor
from torch.utils.data import DataLoader

In [2]:
DATASET_PATH = config.DATA_KAZA
jsonl_file = DATASET_PATH / "data.jsonl"
if jsonl_file.exists():
    df = pd.read_json(jsonl_file, lines=True)
    if "traffic_sequence" in df.columns:
        df = pd.DataFrame(df["traffic_sequence"].tolist())
else:
    df = pd.read_parquet(DATASET_PATH / "data.parquet")
df = df.fillna(0)
X_STEP, Y_STEP = 2, 1
data_array = df.values

In [4]:
PCA_COMPONENTS = 40
dataset = TrafficDataset(data_array, X_STEP, Y_STEP, pca_components=PCA_COMPONENTS)
predictor = TrafficPredictor('lstm', dataset.sensors, X_STEP, Y_STEP, pca_model=dataset.pca if True else None)
predictor.load_model(config.PROJECT_ROOT.joinpath('models/lstm_kaza_pca.pt'))
loader = DataLoader(dataset, batch_size=10, num_workers=8)
preds = predictor.inference(loader)
display(pd.DataFrame(preds.squeeze()).head().iloc[:, :10])


,0,1,2,3,4,5,6,7,8,9
0,-8.405371,3.34346,-0.049417,0.376798,0.287873,-0.706731,-0.130135,-0.02631,-0.301727,-0.110695
1,-8.405371,3.34346,-0.049417,0.376798,0.287873,-0.706731,-0.130135,-0.02631,-0.301727,-0.110695
2,-8.405371,3.34346,-0.049417,0.376798,0.287873,-0.706731,-0.130135,-0.02631,-0.301727,-0.110695
3,-8.405371,3.34346,-0.049417,0.376798,0.287873,-0.706731,-0.130135,-0.02631,-0.301727,-0.110695
4,-8.405371,3.34346,-0.049417,0.376798,0.287873,-0.706731,-0.130135,-0.02631,-0.301727,-0.110695


In [7]:
print(preds[:5])
preds.shape

[[[-8.405371    3.3434598  -0.04941683  0.37679762  0.2878729
   -0.7067312  -0.13013536 -0.02631049 -0.30172712 -0.11069512
    0.07407692 -0.24573076 -0.35003006 -0.10975049 -0.05261604
   -0.26234308  0.08108974 -0.05541497  0.07344048 -0.05953803
   -0.09101108  0.04235891  0.14930427 -0.01860731  0.04933039
   -0.06108296  0.11746143  0.05964286 -0.02095812  0.07936116
    0.02416149  0.06763982 -0.05579697 -0.1153426   0.03512821
   -0.01868577  0.08967444 -0.02229066  0.04625243 -0.06997535]]

 [[-8.405371    3.3434598  -0.04941683  0.37679762  0.2878729
   -0.7067312  -0.13013536 -0.02631049 -0.30172712 -0.11069512
    0.07407692 -0.24573076 -0.35003006 -0.10975049 -0.05261604
   -0.26234308  0.08108974 -0.05541497  0.07344048 -0.05953803
   -0.09101108  0.04235891  0.14930427 -0.01860731  0.04933039
   -0.06108296  0.11746143  0.05964286 -0.02095812  0.07936116
    0.02416149  0.06763982 -0.05579697 -0.1153426   0.03512821
   -0.01868577  0.08967444 -0.02229066  0.04625243 -0.

(283891, 1, 40)